<a href="https://colab.research.google.com/github/VictorCioce/VictorCioce/blob/main/LagoaPatos_visualizacao.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Última atualização: 17/01/2022

Este script irá usar o Google Earth Engine, por Python API, visando acessar imagens da Coleção TOA (Top of Atmosphere) do Landsat 5 e 8. O Foco é visuzalizar A Lagoa dos Patos - RS. Esse algoritmo se propõem a:
 - Acessa as coleções TOA do Landsat 5 e 8;
 - Selecionar imagens por Orbita 221 cena 081/082, Data e Cobertura de nuvens;
 - Criar um Mosaico com as imagens de mesma data em cenas diferentes(81/82);
 - Recorte das imagens por uma camada mascara - Lagoa dos Patos - RS (coordenadas definidas).
 - Visualização das imagens pela lib Folium.

# Instalação e Inicialização do Earth Engine

In [1]:
# Instalando API do Google Earth Engine
!pip install earthengine-api


In [2]:
# Importanto o Earth Engine (ee) e autenticando
import ee
ee.Authenticate()
ee.Initialize()


To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://accounts.google.com/o/oauth2/auth?client_id=517222506229-vsmmajv00ul0bs7p89v5m89qs8eb9359.apps.googleusercontent.com&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fearthengine+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdevstorage.full_control&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&response_type=code&code_challenge=8Y11Tf2fdU_YdPs9gm7-Gu-nahiAIr2bGGHYsDkij2g&code_challenge_method=S256

The authorization workflow will generate a code, which you should paste in the box below. 
Enter verification code: 4/1AX4XfWjEZ5jJfsYw_nZW2gscJyMFG8rLmNH_ynbAOwT0J-wruF81hPg3xgk

Successfully saved authorization token.


# Declaração das Coordenadas da feição da Lagoa dos Patos - RS

In [13]:
# Definindo uma feição simples do entorno da Lagoa dos Patos - RS
feicao_lagoa = ee.Geometry.Polygon([[-50.9847, -30.3578], [-50.9560, -30.3147], 
                             [-50.6642, -30.1425], [-50.5015, -30.1329], 
                             [-50.5207, -30.4582], [-50.6259, -30.5204], 
                             [-50.6977, -30.8218], [-50.9225, -30.9223], 
                             [-50.9321, -31.0371], [-51.1521, -31.1232], 
                             [-51.1378, -31.3863], [-51.6209, -31.8264], 
                             [-51.9797, -31.9029], [-52.0180, -32.2330], 
                             [-52.1615, -32.2091], [-52.3146, -31.9938], 
                             [-52.1567, -31.4485], [-51.5683, -31.0562], 
                             [-51.3052, -30.4199], [-51.3722, -29.9894], 
                             [-51.2526, -29.9655], [-51.1952, -30.1568], 
                             [-51.0134, -30.2525], [-50.9847, -30.3578]])

# Declarando as coordenadas da Lagoa dos Patos como uma Features para recorte
lagoa_patos = ee.Feature(feicao_lagoa)


# Definindo a Função ymdList para visualização de datas

In [5]:
# Função para visualizar as datas das imagens
def ymdList(imgcol):
    def iter_func(image, newlist):
        date = ee.Number.parse(image.date().format("YYYYMMdd"));
        newlist = ee.List(newlist);
        return ee.List(newlist.add(date).sort())
    ymd = imgcol.iterate(iter_func, ee.List([]))
    return list(ee.List(ymd).reduce(ee.Reducer.frequencyHistogram()).getInfo().keys())


# Definição da Função Folium para gerar Mapas da Lagoa dos Patos

In [15]:
# Função para gerar mapa de visualização Folium.
import folium

def add_ee_layer(self, ee_image_object, vis_params, name):
  map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
  folium.raster_layers.TileLayer(
      tiles=map_id_dict['tile_fetcher'].url_format,
      attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
      name=name,
      overlay=True,
      control=True
  ).add_to(self)

folium.Map.add_ee_layer = add_ee_layer

# Definindo parametros da composição do mapa RGB True Color
rgb_param = {
    'bands': ['B3', 'B2', 'B1'],
    'min': 0,
    'max': 0.5,
    'gamma': [0.95, 1.1, 1]
}

# Defindo o centro do mapa para a Lagos dos Patos por Folium 
my_map = folium.Map(location=[-31.15666425, -51.35544453], zoom_start=8)


# 1°Seção A: Filtração e Visualização para TOA Images Landsat 5



Vizualização por USGS Landsat 8 Collection 1 Tier 1 TOA Reflectance

In [10]:
# Variável para acessar coleções do Landsat 5 (LT05_"ROW")
# Filtros Escolhidos para aplicação:
#   Data: 2000-06-23 a 2011-10-28
#   Path: 221/ Row: 081 e 082
#   Cobertura de nuvens menor de 20%

LT05_81 = ee.ImageCollection("LANDSAT/LT05/C01/T1_TOA")\
    .filterDate('2000-06-23','2011-10-28')\
    .filter(ee.Filter.eq('WRS_PATH', 221))\
    .filter(ee.Filter.eq('WRS_ROW', 81))\
    .filterMetadata('CLOUD_COVER', 'less_than', 20)

LT05_82 = ee.ImageCollection("LANDSAT/LT05/C01/T1_TOA")\
    .filterDate('2000-06-23','2011-10-28')\
    .filter(ee.Filter.eq('WRS_PATH', 221))\
    .filter(ee.Filter.eq('WRS_ROW', 82))\
    .filterMetadata('CLOUD_COVER', 'less_than', 20)

print('Total de imagens Row81: ', LT05_81.size().getInfo())
print('Total de imagens Row82: ', LT05_82.size().getInfo())


Total de imagens Row81:  52
Total de imagens Row82:  59


In [11]:
# Usando a função ymdList para visualização de Datas
print('Datas disponíveis ROW 081: ', ymdList(LT05_81))
print('Datas disponíveis ROW 082: ', ymdList(LT05_82))


Datas disponíveis ROW 081:  ['20000623', '20000927', '20010101', '20010525', '20010712', '20010728', '20011219', '20020120', '20020221', '20020309', '20030904', '20031123', '20040211', '20040330', '20040517', '20040704', '20040720', '20040906', '20041008', '20050112', '20050418', '20050504', '20051112', '20060523', '20060608', '20060624', '20061014', '20061115', '20061201', '20070102', '20070203', '20070323', '20070526', '20071001', '20080206', '20080309', '20080325', '20081003', '20081206', '20081222', '20090107', '20090123', '20090328', '20090413', '20090429', '20090702', '20090920', '20100211', '20100822', '20101110', '20110505', '20110910']
Datas disponíveis ROW 082:  ['20000623', '20000927', '20010101', '20010202', '20010407', '20010525', '20010712', '20011117', '20011219', '20020120', '20020309', '20030819', '20030920', '20031123', '20040126', '20040211', '20040501', '20040720', '20040906', '20041024', '20041227', '20050112', '20050128', '20050418', '20050504', '20050621', '20050

In [24]:
# Selecionando imagens por datas da coleção criada a cima
LT05_81 = ee.Image("LANDSAT/LT05/C01/T1_TOA/LT05_221081_20010525")
LT05_82 = ee.Image("LANDSAT/LT05/C01/T1_TOA/LT05_221082_20010525")

# Gerando a variável mosaico
LT05_mosaic = ee.ImageCollection([LT05_81, LT05_82]).mosaic()

# Função Clip que contem o recorte do mosaico sobre a área da Lagoa dos Patos -RS
LT05_LagoaPatos = LT05_mosaic.clip(lagoa_patos)

# # Adicionando as imagens ao MapLayer por Folium.
my_map.add_ee_layer(LT05_LagoaPatos, rgb_param, 'true color')
display(my_map)


# 1°Seção B: Filtração e Visualização para TOA Images Landsat 8; 

In [17]:
# Variável para acessar coleções do Landsat 8 (LC08_"ROW")
# Filtros Escolhidos para aplicação:
#   Data: 2013-08-30 a 2020-12-23
#   Path: 221/ Row: 081 e 082
#   Cobertura de nuvens menor de 20%

LC08_81 = ee.ImageCollection("LANDSAT/LC08/C01/T1_TOA")\
    .filterDate('2013-08-30','2020-12-23')\
    .filter(ee.Filter.eq('WRS_PATH', 221))\
    .filter(ee.Filter.eq('WRS_ROW', 81))\
    .filterMetadata('CLOUD_COVER', 'less_than', 20)

LC08_82 = ee.ImageCollection("LANDSAT/LC08/C01/T1_TOA")\
    .filterDate('2013-08-30','2020-12-23')\
    .filter(ee.Filter.eq('WRS_PATH', 221))\
    .filter(ee.Filter.eq('WRS_ROW', 82))\
    .filterMetadata('CLOUD_COVER', 'less_than', 20)

print('Total de imagens Row81: ', LC08_81.size().getInfo())
print('Total de imagens Row82: ', LC08_82.size().getInfo())


Total de imagens Row81:  55
Total de imagens Row82:  49


In [18]:
# Usando a função ymdList para visualização de Datas
print('Datas disponíveis ROW 081: ', ymdList(LC08_81))
print('Datas disponíveis ROW 082: ', ymdList(LC08_82))


Datas disponíveis ROW 081:  ['20130830', '20131118', '20131204', '20140105', '20140121', '20140206', '20140427', '20141020', '20141207', '20141223', '20150124', '20150225', '20151108', '20160127', '20160212', '20160315', '20160416', '20160518', '20160822', '20161009', '20161212', '20170113', '20170302', '20170403', '20170622', '20170708', '20170724', '20170825', '20171113', '20171215', '20171231', '20180217', '20180422', '20180508', '20180524', '20180609', '20180828', '20180929', '20181015', '20181116', '20190220', '20190324', '20190409', '20190815', '20191002', '20191119', '20200106', '20200207', '20200326', '20200529', '20200614', '20200801', '20200817', '20200918', '20201121']
Datas disponíveis ROW 082:  ['20130830', '20131001', '20131118', '20131204', '20140105', '20140121', '20140222', '20140427', '20141020', '20141105', '20141207', '20150124', '20150209', '20150601', '20151210', '20160212', '20160228', '20160315', '20160619', '20160822', '20161009', '20161212', '20170113', '20170

In [22]:
# Selecionando imagens por datas da coleção criada a cima
LC08_81 = ee.Image("LANDSAT/LC08/C01/T1_TOA/LC08_221081_20130830")
LC08_82 = ee.Image("LANDSAT/LC08/C01/T1_TOA/LC08_221082_20130830")

# Gerando a variável mosaico
LC08_mosaic = ee.ImageCollection([LC08_81, LC08_82]).mosaic()

# Função Clip para recortar o mosaico pela áreas de interesse
LC08_LagoaPatos = LC08_mosaic.clip(lagoa_patos)

# Adicionando as imagens ao MapLayer por Folium
my_map.add_ee_layer(LC08_LagoaPatos, rgb_param, 'true color')
display(my_map)


# FIM DO CÓDIGO!